In [ ]:
# !pip install scikit-surprise 설치시 처음에 에러발생
# Microsoft Visual C++ 14.0 is required. Get it with "Build Tools for Visual Studio      에러 발생. 
# https://visualstudio.microsoft.com/ko/vs/older-downloads/    에서 < Microsoft Visual C++ 2015 재배포 가능 패키지 업데이트 3 >  설치하기

In [2]:
# !pip install scikit-surprise

In [ ]:
# svd 이용(유사도 계산 방법 차이) // 협력필터링 또는 컨텐츠기반 알고리즘으로 구현

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('data-files/movies/movies_metadata.csv')

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [10]:
# 임계치값
print(df['vote_count'].min(), df['vote_count'].max())
m = df['vote_count'].quantile(0.8)    #quantile : 백분위
m    

0.0 14075.0


50.0

In [18]:
#  평점
print(df['vote_average'].min(), df['vote_average'].max())
C = df['vote_average'].mean() 
C

0.0 10.0


5.618207215134185

In [20]:
#  x = df
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return (v/(v+m) * R) + (m/(m+v) * C)

In [23]:
q_movies = df[df['vote_count'] >= m]   #임계치 50건 이상은 되는것 만 뽑기

In [24]:
df.shape , q_movies.shape

((45466, 24), (9151, 24))

In [26]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

c:\programdata\miniconda3\envs\hoseo\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
q_movies[['vote_average','score']]

,vote_average,score
0,7.7,7.680953
1,6.9,6.873979
2,6.5,6.189510
4,5.7,5.681661
5,7.7,7.646235
...,...,...
45339,5.2,5.345211
45343,4.0,4.372859
45348,3.4,4.456289
45437,8.3,7.615869


In [31]:
# apply 는 행별로 적용하라는 의미.
# q_movies.apply(lambda row: row['vote_count'], axis=1)

q_movies=q_movies.sort_values(['score'], ascending=False)

In [32]:
q_movies[['title','vote_count','vote_average','score']].head(20)

,title,vote_count,vote_average,score
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.855148
314,The Shawshank Redemption,8358.0,8.5,8.482863
834,The Godfather,6024.0,8.5,8.476278
40251,Your Name.,1030.0,8.5,8.366584
12481,The Dark Knight,12269.0,8.3,8.289115
2843,Fight Club,9678.0,8.3,8.286216
292,Pulp Fiction,8670.0,8.3,8.284623
522,Schindler's List,4436.0,8.3,8.270109
23673,Whiplash,4376.0,8.3,8.269704
5481,Spirited Away,3968.0,8.3,8.266628
